In [ ]:
# this notebook is used to modeling a powell lens
# the powell lens is a kind of lens that can generate a line light source


In [ ]:
# import the necessary packages
%pylab inline
import os
import pyoptic2 as pyo
from pyoptic2.util import zemax, read_agf
from pyoptic2.display import plot2d, plot_system2d
import pandas as pd
from pyoptic2 import material
# get the current path
mysetup_dir = os.getcwd()
# define the .zar file name
zar_file_name1 = 'AC254_100_A_Zemax.zar'
zar_file_name2 = 'LJ1567L1_Zemax.zar'
# combine the directory and the file name
zar_file_path1 = os.path.join(mysetup_dir, zar_file_name1)
print(zar_file_path1)
zar_file_path2 = os.path.join(mysetup_dir, zar_file_name2)
print(zar_file_path2)
# read .zar file
AC254_100_A = zemax.read_cached_zar(zar_file_path1)
LJ1567L1= zemax.read_cached_zar(zar_file_path2)

# add materials
air = material.Material(material.Material.REFRACT, 1.0)
# add zemax glasses
glasses = zemax.readZar(zar_file_path2)[1] # Get F_SILICA glass from LJ1567L1_Zemax.zar
# build a dictionary of glasses and convert the zemax data to glass dictionary
lens_glasses = {}
for i in glasses.keys():
    glass_data = glasses.get(i, None)
    g = zemax.Glass(glass_data)
    lens_glasses[i] = g
        
lens_glasses['air'] = air
print(lens_glasses['F_SILICA'].__dict__)

In [ ]:
# create a new light source
src = pyo.ColimatedSource('light source', pyo.Placement([0,0,0], [0,1,0]), diameter=30, color=(1,0,0), wavelength=488.
)
r = src.principle_ray #get the on-axis ray from the light source

# generate the system (basically just a container for surfaces)
s = pyo.System()


In [ ]:
# add surfaces
# placement orientation is [x,y,z], y is the optical axis, the unit is 0.1mm
s.append(pyo.PlaneSurface(
    name= 'source plane',
    dimension=np.ones(3) * 100.,
    placement=pyo.Placement([0,0,0], [0,1,0]),
    material=lens_glasses['air'],
))

s.append(pyo.SphericalSurface(
    name='powell lens',
    dimension=np.ones(3) * 16.,
    placement=pyo.Placement([0,30,0], [0,1,0]),
    curvature_radius=50,
    material=lens_glasses['BK7'],
))


s.append(pyo.PlaneSurface(
    name= 'detector',
    dimension=np.ones(3) * 5.,
    placement=pyo.Placement([26, 220, 0], [0, 1, 0]),
    material=lens_glasses['air'],
))




In [ ]:
# plot the 2d system
figure(figsize=(18,5))
plot_system2d(s, 'xy')
plot2d(s.propagate(src.chief_rays), 'xy')
axis('equal')
grid()